# Computation of layer by layer containment corrections




## Setup

In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load ../drawingTools


In [3]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v87t'
version_V8 = version
version_V9 = 'v107'
version_V10 = 'v106'


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0', 'PU0', version_V8, 'V8'),
    Sample('ele_flat2to100_PU200', 'PU200', version_V8, 'V8')
    ]


samples_ele_V9 = [
    Sample('ele_flat2to100_PU0_calibs', 'PU0', version_V9, 'ele-V9'),
    Sample('ele_flat2to100_PU200_calibs', 'PU200', version_V9, 'ele-V9')
    ]

samples_ele_V10 = [
    Sample('ele_flat2to100_PU0_calibs', 'PU0', version_V10, 'ele-V10'),
    Sample('ele_flat2to100_PU200_tps', 'PU200', version_V10, 'ele-V10')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0_tps', 'PU0', version, 'photon'),
#     Sample('photon_flat8to150_PU200_tps', 'PU200', version, 'photon')
    ]

samples_photons_V8 = [
    Sample('photon_flat8to150_PU0', 'PU0', version_V8, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version_V8, 'photon')
    ]

samples_photons_V9 = [
    Sample('photon_flat8to150_PU0_calibs', 'PU0', version_V9, 'photons-V9'),
#     Sample('photon_flat8to150_PU200_tps', 'PU200', version_V9, 'photons-V9')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates_V8 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V8')
    ]

samples_nugunrates_V9 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V9')
    ]


gen_part_selections: 14


Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU0_v87t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU200_v87t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU0_v87t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU200_v87t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU200_tps_v106.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU0_tps_v87t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_pion_flat2to100_PU0_v87t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu0_v87t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v87t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v87t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v87t.roo

In [4]:
# %load settings.py
# %load settings.py
# from samples import get_label_dict
import python.selections as selections
import python.collections as collections

import pprint



def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary



# === samples =====================================================
samples = []

# samples += samples_photons_V9
samples += samples_ele_V9
# samples += samples_ele_V10


sample = 'ele-V9'

do_rate = False

# === TP ==========================================================
# tps = ['EG', 'TkEG', 'TkEle', 'TkIsoEle']
# tps = ['HMvDR', 'HMvDRcylind10', 'HMvDRcylind5', 'HMvDRcylind2p5']

tps = ['HMvDR',
       'HMvDRshapeDr']
#        'HMvDRcylind5', 'HMvDRcylind2p5', 'HMvDRshapeCalib', 'HMvDRshapeDrCalib',
#        'HMvDRcylind5Calib', 'HMvDRcylind2p5Calib']


# === TP selections ================================================
tp_select = {}
tp_select_default = ['Em', 'all']

tp_select['EG'] = ['EGq2', 'EGq3']

tp_select['TkEG'] = get_label_dict(selections.tkeg_qual_selections).keys()
tp_select['TkEle'] = ['EGq2', 'EGq3']

tp_select['TkEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['TkIsoEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['L1Trk'] = ['all', 'Pt2', 'Pt10']

for tp in tps:
    if not tp in tp_select.keys():
        tp_select[tp] = tp_select_default

# ==== GEN selections ===============================================

gen_select = ['GENEtaBC', 'GENEtaD']
# gen_select = ['GEN', 'GENEtaA', 'GENEtaB', 'GENEtaC', 'GENEtaD', 'GENEtaE',
#               'GENEtaAB', 'GENEtaABC', 'GENEtaBC', 'GENEtaBCD', 'GENEtaBCDE' ]

# tp_select['TkEG'] = ['EGq2EtaBC', 'EGq3EtaBC', 'EGq2EtaBCM2', 'EGq3EtaBCM2', 'EGq2EtaBCM2s', 'EGq3EtaBCM2s','EGq2EtaBCM3', 'EGq3EtaBCM3', 'EGq2EtaBCM3s', 'EGq3EtaBCM3s','EGq2EtaBCM4', 'EGq3EtaBCM4']



event_manager = collections.EventManager()

all_tpsets = {}
for collection in event_manager.collections:
    all_tpsets[collection.name] = collection.label


# all_tpsets = {'DEF': 'dRC3d',
#               'DEFCalib': 'NNDR Calib v1',
#               'DEFNC': 'dRC3d + new Th',
#               'HMvDR': 'HistoMaxC3d + dR(layer)',
#               'HMvDRNC0': 'HMC3d+dR(layer)+NC0',
#               'HMvDRNC1': 'HMC3d+dR(layer)+NC1',
#               'EG': 'EG',
#               'TkEG': 'TkEG',
#               'TkEle': 'TkEle',
#               'TkIsoEle': 'TkIsoEle',
#               'L1Trk': 'L1Track'}


# ==== adapt the plot retrieval
tpsets = {}
tpset_selections = {}
gen_selections = {}

for tp in tps:
    tpsets[tp] = all_tpsets[tp]

if 'DEF' in tps or 'HMvDR' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tp_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tp_match_selections))
if 'EG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.eg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.eg_pt_selections))
if 'TkEG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkeg_pt_selections))
if 'TkEle' in tps or 'TkIsoEle' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkisoeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkisoeg_pt_selections))
if 'L1Trk' in tps:
    tpset_selections.update(get_label_dict(selections.tracks_selections))


gen_selections.update(get_label_dict(selections.gen_part_selections_calib))
# gen_selections.update({'nomatch': ''})


pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
print gen_select


[EventManager] registering collection: MC
[EventManager] registering collection: GEN
[EventManager] registering collection: TC
[EventManager] registering collection: TCTrue
[EventManager] registering collection: DEF2D
[EventManager] registering collection: DEF2DTrue
[EventManager] registering collection: HMvDRTrue
[EventManager] registering collection: DEF
[EventManager] registering collection: DEFNC
[EventManager] registering collection: HMvDR
[EventManager] registering collection: HMvDREmInt
[EventManager] registering collection: HMvDREmIntMerged
[EventManager] registering collection: HMvDRRebin
[EventManager] registering collection: HMvDRsTC
[EventManager] registering collection: HMvDRNC0
[EventManager] registering collection: HMvDRNC1
[EventManager] registering collection: DEFMerged
[EventManager] registering collection: DEFCalib
[EventManager] registering collection: HMvDRMerged
[EventManager] registering collection: HMvDRfixed
[EventManager] registering collection: HMvDRcylind10


In [5]:
tpsets


{'HMvDR': 'HM+dR(layer) Cl3d', 'HMvDRshapeDr': 'HM #Delta#rho < 0.015'}

In [6]:
tpset_selections

{'Em': 'EGId',
 'EmPt10': 'EGId, p_{T}^{L1}>=10GeV',
 'EmPt20': 'EGId, p_{T}^{L1}>=20GeV',
 'EmPt30': 'EGId, p_{T}^{L1}>=30GeV',
 'Pt10': 'p_{T}^{L1}>=10GeV',
 'Pt20': 'p_{T}^{L1}>=20GeV',
 'Pt30': 'p_{T}^{L1}>=30GeV',
 'all': ''}

In [7]:
gen_selections

{'GEN': '',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaBCDHBrem': '1.52 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5',
 'GENEtaBCDLBrem': '1.52 < |#eta^{GEN}| <= 2.8, f_{BREM} < 0.5',
 'GENEtaBCHBrem': '1.52 < |#eta^{GEN}| <= 2.4, f_{BREM} >= 0.5',
 'GENEtaBCLBrem': '1.52 < |#eta^{GEN}| <= 2.4, f_{BREM} < 0.5',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENEtaDHBrem': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5',
 'GENEtaDHBremPt10': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=10GeV',
 'GENEtaDHBremPt20': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=20GeV',
 'GENEtaDHBremPt30': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=30GeV',
 'GENEtaDHBremPt40': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=40GeV',
 'GENEtaDLBrem': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} < 0.5'}

In [8]:
from python.plotters import tp_calib_plotters

histo_primitives = pd.DataFrame() 

for plotter in tp_calib_plotters:
    histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)

    
print histo_primitives.data.unique()

['HMvDR' 'HMvDRCalib' 'HMvDREmInt' 'HMvDREmIntMerged' 'HMvDRshapeDr'
 'HMvDRcylind5' 'HMvDRcylind2p5' 'HMvDRshapeCalib' 'HMvDRshapeDrCalib'
 'HMvDRcylind5Calib' 'HMvDRcylind2p5Calib']


In [9]:
samples

[<drawingTools.Sample instance at 0x1230c5128>,
 <drawingTools.Sample instance at 0x1230c5248>]

In [10]:
files

{}

In [37]:
class EnergyRes(histos.ResoHistos):
    def __init__(self,  name, root_file=None, debug=False):
        self.name = name
        self.h_energyResVenergy = ROOT.TH2F('{}_energyResVenergy'.format(self.name), '', 100,0,1000, 200, -100, 100)
        self.h_energyResp = ROOT.TH1F('{}_energyResp; E^{L1}/E^{GEN}'.format(self.name), '', 200, 0, 2)

        self.h_lCalibEnergyResVenergy = ROOT.TH2F('{}_lCalibEnergyResVenergy'.format(self.name), '', 100,0,1000, 200, -100, 100)
        self.h_lCalibEnergyResp = ROOT.TH1F('{}_lCalibEnergyResp; E^{L1}/E^{GEN}'.format(self.name), '', 200, 0, 2)

        self.h_etaEnergyResVenergy = ROOT.TH2F('{}_etaEnergyResVenergy'.format(self.name), '', 100,0,1000, 200, -100, 100)
        self.h_etaEnergyResp = ROOT.TH1F('{}_etaEnergyResp; E^{L1}/E^{GEN}'.format(self.name), '', 200, 0, 2)

        self.h_etalCalibEnergyResVenergy = ROOT.TH2F('{}_etalCalibEnergyResVenergy'.format(self.name), '', 100,0,1000, 200, -100, 100)
        self.h_etalCalibEnergyResp = ROOT.TH1F('{}_etalCalibEnergyResp; E^{L1}/E^{GEN}'.format(self.name), '', 200, 0, 2)

        histos.BaseResoHistos.__init__(self, name, root_file)
    
    def fill(self, tree, expression):
        hname = '{}_energyResVenergy'.format(self.name)
        draw_expression = '{}>>{}(100,0,1000, 200, -100, 100)'.format(expression, hname)
        tree.Draw(draw_expression,"","COL")
        self.h_energyResVenergy = ROOT.gPad.GetPrimitive(hname)
        
        
    def fill_df(self, df):
        rnp.fill_hist(self.h_energyResVenergy, df[['gen_energy', 'uncalib_res']])
        rnp.fill_hist(self.h_lCalibEnergyResVenergy, df[['gen_energy', 'lcalib_res']])
        rnp.fill_hist(self.h_etaEnergyResVenergy, df[['gen_energy', 'eta_uncalib_res']])
        rnp.fill_hist(self.h_etalCalibEnergyResVenergy, df[['gen_energy', 'eta_lcalib_res']])
  
        rnp.fill_hist(self.h_energyResp, df.uncalib_energy_nl0/df.gen_energy)
        rnp.fill_hist(self.h_lCalibEnergyResp, df.lcalib_energy/df.gen_energy)
        rnp.fill_hist(self.h_etaEnergyResp, df.eta_uncalib_energy_nl0/df.gen_energy)
        rnp.fill_hist(self.h_etalCalibEnergyResp, df.eta_lcalib_energy/df.gen_energy)

        
class HWrapper(object):
    def __init__(self, histo):
        self.histo = histo
    
    def get(self, debug=False):
        return self.histo


## Computation of the values

In [12]:
calib_table = {}
# calib_table['HMvDRfixedCalib'] = [4.5, 0.94, 1.08, 1.04, 0.91, 1.04, 1.09, 0.91, 1.0, 0.91, 1.06, 1.07, 1.37, 1.47]
# calib_table['HMvDRcylind10Calib'] = [4.5, 0.94, 1.08, 1.04, 0.91, 1.04, 1.09, 0.91, 1.0, 0.91, 1.06, 1.07, 1.37, 1.47]
# calib_table['HMvDRcylind5Calib'] = [4.91, 0.94, 1.09, 1.05, 0.92, 1.05, 1.11, 0.94, 1.02, 0.92, 1.1, 1.04, 1.34, 1.6]
# calib_table['HMvDRcylind2p5Calib'] = [8.49, 0.97, 1.19, 1.18, 1.04, 1.1, 1.25, 1.05, 1.14, 0.97, 1.17, 1.34, 1.3, 1.67]
#                                      [7.86, 1.1, 1.23, 1.0, 1.18, 1.23, 0.99, 1.15, 1.11, 1.16, 0.98, 1.4, 1.47, 1.45]

# # v89
# calib_table['HMvDRfixedCalib'] = [4.52, 0.93, 1.08, 1.04, 0.91, 1.04, 1.09, 0.91, 1.0, 0.9, 1.07, 1.06, 1.36, 1.47]
# calib_table['HMvDRcylind10Calib'] = [4.53, 0.88, 1.14, 1.0, 0.92, 1.06, 1.07, 0.89, 1.06, 0.83, 1.07, 1.13, 1.34, 1.49]
# calib_table['HMvDRcylind5Calib'] = [4.98, 1.03, 0.98, 1.12, 0.91, 1.05, 1.14, 0.87, 1.02, 0.97, 1.13, 1.01, 1.43, 1.4]
# calib_table['HMvDRcylind2p5Calib'] = [7.86, 1.1, 1.23, 1.0, 1.18, 1.23, 0.99, 1.15, 1.11, 1.16, 0.98, 1.4, 1.47, 1.45]
# calib_table['HMvDRshapeCalib'] = [1]*28

# # v93
# calib_table['HMvDRCalib'] = [6.38, 0.98, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.81, 1.0, 0.9, 1.08, 1.5, 1.81]
# calib_table['HMvDRcylind10Calib'] = [6.39, 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]
# calib_table['HMvDRcylind5Calib'] = [6.92, 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.06, 0.83, 1.04, 0.92, 1.08, 1.5, 1.89]
# calib_table['HMvDRcylind2p5Calib'] = [12.35, 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15]
# calib_table['HMvDRshapeCalib'] = [8.43, 1.69, 1.1, 1.2, 0.97, 0.97, 1.13, 1.03, 0.82, 1.0, 0.89, 1.08, 1.55, 1.82]
# calib_table['HMvDRshapeDrCalib'] = [1.]*14

# # v96
# calib_table['HMvDRCalib'] = [6.38, 0.98, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.81, 1.0, 0.9, 1.08, 1.5, 1.81]
# calib_table['HMvDRcylind10Calib'] = [6.39, 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]
# calib_table['HMvDRcylind5Calib'] = [6.92, 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.06, 0.83, 1.04, 0.92, 1.08, 1.5, 1.89]
# calib_table['HMvDRcylind2p5Calib'] = [12.35, 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15]
# calib_table['HMvDRshapeCalib'] = [7.17, 1.38, 1.05, 1.07, 0.96, 0.97, 1.11, 1.04, 0.81, 1.02, 0.9, 1.07, 1.52, 1.84]
# calib_table['HMvDRshapeDrCalib'] = [6.96, 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.05, 0.83, 1.03, 0.91, 1.07, 1.51, 1.89]

# v96bis
# calib_table['HMvDRCalib'] = [7.16, 2.03, 1.06, 0.75, 1.24, 0.96, 0.83, 1.15, 1.06, 0.87, 0.86, 0.93, 1.24, 2.59]
# calib_table['HMvDRcylind10Calib'] = [1., 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]
# calib_table['HMvDRcylind5Calib'] = [1., 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.06, 0.83, 1.04, 0.92, 1.08, 1.5, 1.89]
# calib_table['HMvDRcylind2p5Calib'] = [1., 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15]
# calib_table['HMvDRshapeCalib'] = [1., 1.38, 1.05, 1.07, 0.96, 0.97, 1.11, 1.04, 0.81, 1.02, 0.9, 1.07, 1.52, 1.84]
# calib_table['HMvDRshapeDrCalib'] = [7.43, 2.14, 1.06, 0.84, 1.14, 1.0, 0.99, 1.0, 1.07, 0.95, 0.9, 0.91, 1.19, 2.71]


calib_table['HMvDRCalib'] = [0.0, 2.64, 0.95, 0.76, 1.19, 0.98, 0.85, 1.15, 1.04, 0.86, 0.87, 0.93, 1.27, 2.56]
calib_table['HMvDRcylind10Calib'] = [1., 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]
calib_table['HMvDRcylind5Calib'] = [1., 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.06, 0.83, 1.04, 0.92, 1.08, 1.5, 1.89]
calib_table['HMvDRcylind2p5Calib'] = [1., 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15]
calib_table['HMvDRshapeCalib'] = [1., 1.38, 1.05, 1.07, 0.96, 0.97, 1.11, 1.04, 0.81, 1.02, 0.9, 1.07, 1.52, 1.84]
# calib_table['HMvDRshapeDr'] = [0., 1.6144949, 0.92495334, 1.0820811, 0.9753549, 0.9742881, 1.0634482, 1.0599478, 0.9376349, 0.92587173, 0.8003076, 1.0417082, 1.7032381, 2.]
# calib_table['HMvDRshapeDrCalib'] = [0., 2.0, 1.35, 0.82, 1.03, 1.01, 1.01, 1.03, 1.07, 0.91, 0.87, 1.08, 1.42, 1.99]
# calib_table['HMvDRshapeDrCalib'] = [2.0, 1.47, 0.96, 1.09, 0.95, 1.09, 1.02, 0.99, 0.94, 1.1, 1.21, 0.96, 1.33, 1.93]
calib_table['HMvDRshapeDrCalib'] =  [0.] + [1.]*13

# calib_table['HMvDR'] = [0.] + [1.]*13
# calib_table['HMvDRcylind10'] =  [0.] + [1.]*13
# calib_table['HMvDRcylind5'] =  [0.] + [1.]*13
# calib_table['HMvDRcylind2p5'] =  [0.] + [1.]*13
# calib_table['HMvDRshape'] =  [0.] + [1.]*13
# calib_table['HMvDRshapeDr'] =  [0.] + [1.]*13


# v102 ele-V10
calib_table_V10 = {}
calib_table_V10['HMvDRCalib'] = [8.04, 1.61, 1.08, 0.8, 1.23, 1.04, 0.91, 1.16, 1.05, 1.04, 0.84, 1.05, 1.36, 2.48]
calib_table_V10['HMvDRcylind10Calib'] = [1., 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]
calib_table_V10['HMvDRcylind5Calib'] = [8.6, 1.63, 1.09, 0.92, 1.12, 1.1, 1.05, 1.03, 1.09, 1.08, 0.94, 0.97, 1.42, 2.59]
calib_table_V10['HMvDRcylind2p5Calib'] = [1., 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15]
calib_table_V10['HMvDRshapeCalib'] = [1., 1.38, 1.05, 1.07, 0.96, 0.97, 1.11, 1.04, 0.81, 1.02, 0.9, 1.07, 1.52, 1.84]
calib_table_V10['HMvDRshapeDrCalib'] = [8.65, 1.63, 1.09, 0.93, 1.11, 1.1, 1.05, 1.03, 1.07, 1.09, 0.94, 0.98, 1.36, 2.65]

print calib_table


calib_table_by_sample = {}
calib_table_by_sample['ele-V10'] = calib_table_V10
calib_table_by_sample['ele-V9'] = calib_table
calib_table_by_sample['photons-V10'] = calib_table_V10
calib_table_by_sample['photons-V9'] = calib_table

{'HMvDRCalib': [0.0, 2.64, 0.95, 0.76, 1.19, 0.98, 0.85, 1.15, 1.04, 0.86, 0.87, 0.93, 1.27, 2.56], 'HMvDRcylind5Calib': [1.0, 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.06, 0.83, 1.04, 0.92, 1.08, 1.5, 1.89], 'HMvDRcylind2p5Calib': [1.0, 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15], 'HMvDRshapeCalib': [1.0, 1.38, 1.05, 1.07, 0.96, 0.97, 1.11, 1.04, 0.81, 1.02, 0.9, 1.07, 1.52, 1.84], 'HMvDRshapeDrCalib': [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'HMvDRcylind10Calib': [1.0, 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]}


In [13]:
tps

['HMvDR', 'HMvDRshapeDr']

In [14]:
# do_calibrations = True

# samples = []
# # samples += samples_photons_V9
# samples += samples_ele_V9
# # samples += samples_ele_V10

# hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)

# class HWrapper(object):
#     def __init__(self, histo):
#         self.histo = histo
    
#     def get(self, debug=False):
#         return self.histo


# from sklearn.linear_model import LinearRegression
# from scipy.optimize import least_squares

# def compute_residual(layer_params, layer_energies, gen_energy):
# #     print 'layer_params: {}'.format(layer_params)
# #     print layer_params.shape
# #     print 'layer_energies: {}'.format(layer_energies)
# #     print layer_energies.shape
# #     print 'gen_energy: {}'.format(gen_energy)
# #     print gen_energy.shape
# #     new_energy = layer_params*layer_energies
# #     print 'layer_params*layer_energies {}'.format(new_energy)
# #     print 'sum {}'.format(np.sum(new_energy, axis=1))
# #     print 'diff {}'.format(np.sum(new_energy, axis=1) - gen_energy)
#     return np.sum(layer_params*layer_energies, axis=1) - gen_energy
    


# import array
# import numpy as np 
# for smp in samples:
#     print ""
#     print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
#     calib_table = calib_table_by_sample[smp.type]
#     for tp in tps:
#         for gen_sel in ['GENEtaBC', 'GENEtaD', 'GENEtaBCD']:

#     #     for gen_sel in gen_selections:
#             if gen_sel == 'nomatch':
#                 continue
#             for tp_sel in ['Em']:
#                 print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)
#                 tree_name = '{}_{}_{}_calib'.format(tp, tp_sel, gen_sel)
#                 if smp.type == 'ele-V9' and tp == "HMvDRCalib":
#                     tree_name = '{}_{}_{}_calib'.format('HMvDR', tp_sel, gen_sel)
#     #             samples[0].histo_file.ls()
# #                 print tree_name
#                 tree = smp.histo_file.Get(tree_name)
# #       tree.Print()
#     #             draw([hres.h_energyResVenergy], labels=[''], text='')
#                 branches = ['e1', 'e3','e5','e7','e9','e11','e13','e15','e17','e19','e21','e23','e25','e27']
#                 fill_expression = 'e1+e3+e5+e7+e9+e11+e13+e15+e17+e19+e21+e23+e25+e27-Egen:Egen'
#                 if tp in calib_table.keys():
#                     calibrated_branches = []
#                     calib_factors = calib_table[tp]
#                     fill_expression = ''

#                     for idx,branch in enumerate(branches):
#                         calibrated_branches.append('{}*{}'.format(branch, calib_factors[idx]))
#                         if idx != 0:
#                            fill_expression += '+' 
#                         fill_expression += '{}*{}'.format(branch, calib_factors[idx])
#                     fill_expression += '-Egen:Egen'
#                     branches = calibrated_branches
#     #             print  fill_expression
#     #             continue
#                 hres = EnergyRes(name='{}_{}_{}'.format(tp, tp_sel, gen_sel))
#                 hres.fill(tree, fill_expression)

#                 hplot.data = hplot.data.append({'sample': smp.type,
#                                        'pu': smp.label,
#                                        'tp': tp,
#                                        'tp_sel': tp_sel,
#                                        'gen_sel': gen_sel,
#                                        'classtype': EnergyRes,
#                                        'histo': HWrapper(hres),},
#                                        ignore_index=True)

        
        
#                 if smp.label == 'PU200':
#                     continue
                    
#                 if not do_calibrations:
#                     continue

#                 pt = np.array([list(aset)[0] for aset in rnp.tree2array(tree, branches=['pt'], cache_size=400000000)])
    
#                 layer_energy_list = np.array([list(aset) for aset in rnp.tree2array(tree, branches=branches, cache_size=400000000)])

# #                 print layer_energy[:10]

#     #             print layer_corr
#                 target_energy = np.array(rnp.tree2array(tree, branches='Egen', cache_size=400000000))
#     #             print layer_energy
            
#                 # here we filter the data according to some cut!
#                 mask = pt < 1000
#                 layer_energy_list = layer_energy_list[mask]
#                 target_energy = target_energy[mask]

#                 if len(layer_energy_list) == 0:
#                     continue
#     #             print layer_energy_list
#     #             print layer_energy_list*layer_corr
#                 regression = LinearRegression(fit_intercept=False)
#                 regression.fit(layer_energy_list, target_energy)
#                 print 'Calibration Name: {} nclusters: {}, score: {}, coefficients:'.format(tree_name, len(layer_energy_list), regression.score(np.array(layer_energy_list), np.array(target_energy)))
                
#                 print '   - lin. reg: {}'.format([float("{0:0.2f}".format(i)) for i in regression.coef_ ])
#                 sumv = 0.
#                 for value in regression.coef_:
#                     sumv+=value
#                 print sumv
#     #                                       start=self._entry, stop=self._entry+1, cache_size=400000000)

#                 res = least_squares(fun=compute_residual, 
#                                     x0=np.array([1.]*14), 
#                                     args=(layer_energy_list, target_energy),
#                                     bounds=(np.array([0.5]*14), np.array([2.]*14)))
#                 print '   - least. sq bound: {}'.format([float("{0:0.2f}".format(i)) for i in res.x ])
        
#                 from scipy.optimize import lsq_linear
#                 residuals = np.sum(layer_energy_list, axis=1)/target_energy
#                 mean_residuals = np.mean(residuals)
#                 meanBounds = 1/mean_residuals
#                 lsqbregr=lsq_linear(layer_energy_list, 
#                                     target_energy, 
#                                     bounds = ((meanBounds)/2.0,(meanBounds)*2), 
#                                     method='bvls', lsmr_tol='auto', verbose=1)
#                 weights = lsqbregr.x
#                 print '   - linear. lsq bound: {}'.format([float("{0:0.2f}".format(i)) for i in lsqbregr.x ])

#                 if gen_sel is 'GENEtaBCD':
#                     abs_eta = np.array([abs(list(aset)[0]) for aset in rnp.tree2array(tree, branches=['eta'], cache_size=400000000)])
# #                     X =  np.sum(layer_energy_list, axis=1)
#                     energy = np.sum(layer_energy_list, axis=1)
#                     y = target_energy-energy
#                     print abs_eta
#                     print y
#                     print target_energy
#                     corr_eta = LinearRegression(fit_intercept=True).fit(abs_eta.reshape(-1,1), y)
#                     print corr_eta.coef_, corr_eta.intercept_

#                 graph = ROOT.TGraph(len(regression.coef_), array.array('d', range(1,28,2)), array.array('d', res.x))
#                 graph.SetMarkerStyle(7)
#                 graph.SetMarkerColor(2)
#                 stuff.append(graph)
#                 draw([graph], labels=['test'], text=tree_name, options='A', min_y=0., max_y=2.)

#                 hplot.data = hplot.data.append({'sample': smp.type,
#                                                'pu': smp.label,
#                                                 'tp': tp,
#                                                 'tp_sel': tp_sel,
#                                                 'gen_sel': gen_sel,
#                                                 'classtype': 'pippo',
#                                                 'histo': HWrapper(graph),},
#                                                 ignore_index=True)


            

### read the calib trees

In [15]:
samples = []
samples += samples_photons_V9
samples += samples_ele_V9
# samples += samples_ele_V10


import array
import numpy as np 

tree_data = pd.DataFrame()

for smp in samples:
    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    calib_table = calib_table_by_sample[smp.type]
    for tp in tps:
        for gen_sel in ['GENEtaBC', 'GENEtaD', 'GENEtaBCD']:
            for tp_sel in ['Em']:
                print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)
                tree_name = '{}_{}_{}_calib'.format(tp, tp_sel, gen_sel)
                if smp.type == 'ele-V9' and tp == "HMvDRCalib":
                    tree_name = '{}_{}_{}_calib'.format('HMvDR', tp_sel, gen_sel)
                # get the tree
                tree = smp.histo_file.Get(tree_name)
                
                # read the branches
                df = pd.DataFrame()

                cluster_pt = np.array([list(aset)[0] for aset in rnp.tree2array(tree, branches=['pt'], cache_size=400000000)])
                cluster_eta = np.array([list(aset)[0] for aset in rnp.tree2array(tree, branches=['eta'], cache_size=400000000)])
                branches = ['e1', 'e3','e5','e7','e9','e11','e13','e15','e17','e19','e21','e23','e25','e27']

                cluster_layer_energies = np.array([np.array(list(aset)) for aset in rnp.tree2array(tree, branches=branches, cache_size=400000000)])
                gen_energy = np.array(rnp.tree2array(tree, branches='Egen', cache_size=400000000))
                
                # fill the array

                df['cl_pt'] = cluster_pt
                df['cl_layer_energies'] = cluster_layer_energies.tolist()
                df['cl_eta'] = cluster_eta
                df['cl_abseta'] = np.abs(df.cl_eta)
                df['gen_energy'] = gen_energy
                # FIXME: approx
                df['gen_pt'] = df.gen_energy/np.cosh(df.cl_eta)
        
                
                df['uncalib_energy'] = df.cl_layer_energies.apply(np.sum)
                df['uncalib_energy_nl0'] = df.cl_layer_energies.apply(lambda x: np.dot(x, [0]+[1.]*13))

                df.loc[:, 'smp'] = smp.type
                df.loc[:, 'pu'] = smp.label
                df.loc[:, 'tp'] = tp
                df.loc[:, 'gen_sel'] = gen_sel
                df.loc[:, 'tp_sel'] = tp_sel

                tree_data = tree_data.append(df, ignore_index=True)



------ SAMPLE: photons-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD

------ SAMPLE: ele-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD

------ SAMPLE: ele-V9, PU: PU200
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD


In [16]:
tree_data.iloc[:10]

,cl_pt,cl_layer_energies,cl_eta,cl_abseta,gen_energy,gen_pt,uncalib_energy,uncalib_energy_nl0,smp,pu,tp,gen_sel,tp_sel
0,17.366413,"[0.0, 3.36193776131, 8.91100883484, 11.7480354...",1.872592,1.872592,67.739655,20.345711,57.821576,57.821576,photons-V9,PU0,HMvDR,GENEtaBC,Em
1,10.426340,"[0.0, 0.77487039566, 3.49941468239, 4.89918041...",-2.141374,2.141374,46.412971,10.757920,44.986223,44.986223,photons-V9,PU0,HMvDR,GENEtaBC,Em
2,40.994465,"[0.0, 0.0, 0.118730135262, 4.19929742813, 15.0...",-1.666960,1.666960,109.525742,39.937462,112.431196,112.431196,photons-V9,PU0,HMvDR,GENEtaBC,Em
3,44.075939,"[0.0, 2.93075966835, 11.7980260849, 20.9090023...",1.706340,1.706340,133.794144,47.025391,125.410269,125.410269,photons-V9,PU0,HMvDR,GENEtaBC,Em
4,70.830299,"[0.0, 0.0, 5.40534591675, 26.289352417, 49.960...",-2.111467,2.111467,294.510956,70.277206,296.844090,296.844090,photons-V9,PU0,HMvDR,GENEtaBC,Em
5,120.486267,"[0.0, 4.87418460846, 21.2589435577, 44.0738754...",1.744260,1.744260,356.694122,120.986252,355.246822,355.246822,photons-V9,PU0,HMvDR,GENEtaBC,Em
6,57.265133,"[0.0, 0.0, 1.66222190857, 14.1101398468, 43.71...",2.349058,2.349058,308.048096,58.280849,302.699369,302.699369,photons-V9,PU0,HMvDR,GENEtaBC,Em
7,11.865688,"[0.0677006542683, 1.22479510307, 5.43659067154...",-1.995320,1.995320,46.951447,12.536216,44.441527,44.373826,photons-V9,PU0,HMvDR,GENEtaBC,Em
8,77.390877,"[0.0, 0.168721780181, 1.89968216419, 15.609889...",-1.674180,1.674180,219.918640,79.653496,213.239322,213.239322,photons-V9,PU0,HMvDR,GENEtaBC,Em
9,108.124046,"[0.0, 0.0, 0.199966549873, 4.04307365417, 20.9...",-1.608215,1.608215,278.339264,107.174461,279.831314,279.831314,photons-V9,PU0,HMvDR,GENEtaBC,Em


### compute the layer-by-layer corrections

In [17]:
# compute the layer calibrations on photons

calib_df = pd.DataFrame()
from sklearn.linear_model import LinearRegression
from scipy.optimize import least_squares
from scipy.optimize import lsq_linear

for smp in samples_photons_V9:
    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    calib_table = calib_table_by_sample[smp.type]
    for tp in tps:
        for gen_sel in ['GENEtaBC', 'GENEtaD', 'GENEtaBCD']:
            for tp_sel in ['Em']:
                print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)

                sel_df = tree_data[(tree_data.smp == smp.type) &
                                   (tree_data.pu == smp.label) &
                                   (tree_data.tp == tp) &
                                   (tree_data.tp_sel == tp_sel) &
                                   (tree_data.gen_sel == gen_sel)]
                
                
#                 res = least_squares(fun=compute_residual, 
#                                     x0=np.array([1.]*14), 
#                                     args=(layer_energy_list, target_energy),
#                                     bounds=(np.array([0.5]*14), np.array([2.]*14)))
#                 print '   - least. sq bound: {}'.format([float("{0:0.2f}".format(i)) for i in res.x ])
                lsqbregr=lsq_linear(np.array(sel_df.cl_layer_energies.tolist()), 
                                    sel_df.gen_energy.to_numpy(), 
                                    bounds = (np.array([0.5]*14), np.array([2.]*14)), 
                                    method='bvls', lsmr_tol='auto', verbose=1)
                weights = lsqbregr.x
                print '   - linear. lsq bound: {}'.format([float("{0:0.2f}".format(i)) for i in lsqbregr.x ])

                df = pd.DataFrame()
                df['layer_calib'] = [lsqbregr.x]
                df.loc[:, 'smp'] = smp.type
                df.loc[:, 'pu'] = smp.label
                df.loc[:, 'tp'] = tp
                df.loc[:, 'gen_sel'] = gen_sel
                df.loc[:, 'tp_sel'] = tp_sel

                calib_df = calib_df.append(df, ignore_index=True)

                


------ SAMPLE: photons-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
The relative change of the cost function is less than `tol`.
Number of iterations 3, initial cost 7.0910e+05, final cost 7.0832e+05, first-order optimality 9.25e-08.
   - linear. lsq bound: [2.0, 1.29, 0.97, 1.07, 0.94, 0.97, 1.09, 1.02, 0.81, 1.0, 0.95, 1.06, 1.49, 1.78]
TP: HMvDR sel: Em, gen_sel: GENEtaD
The relative change of the cost function is less than `tol`.
Number of iterations 4, initial cost 7.1507e+06, final cost 7.0420e+06, first-order optimality 9.01e-08.
   - linear. lsq bound: [2.0, 0.61, 1.9, 1.07, 0.5, 1.2, 1.02, 1.18, 1.15, 0.5, 0.73, 1.45, 1.3, 2.0]
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
The relative change of the cost function is less than `tol`.
Number of iterations 3, initial cost 7.8642e+06, final cost 7.8336e+06, first-order optimality 2.27e-07.
   - linear. lsq bound: [2.0, 0.9, 1.28, 1.07, 0.76, 1.06, 1.06, 1.1, 1.0, 0.75, 0.85, 1.25, 1.3, 2.0]
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC

In [18]:
calib_df.iloc[:10]

,layer_calib,smp,pu,tp,gen_sel,tp_sel
0,"[2.0, 1.29057666452, 0.970505584386, 1.0680388...",photons-V9,PU0,HMvDR,GENEtaBC,Em
1,"[2.0, 0.606425578737, 1.89625355245, 1.0735639...",photons-V9,PU0,HMvDR,GENEtaD,Em
2,"[2.0, 0.900639715236, 1.27626702483, 1.0743271...",photons-V9,PU0,HMvDR,GENEtaBCD,Em
3,"[2.0, 1.32268467149, 0.97932604005, 1.08536347...",photons-V9,PU0,HMvDRshapeDr,GENEtaBC,Em
4,"[2.0, 0.607851491116, 1.89343516653, 1.0939786...",photons-V9,PU0,HMvDRshapeDr,GENEtaD,Em
5,"[2.0, 0.942846467491, 1.28828676553, 1.0986588...",photons-V9,PU0,HMvDRshapeDr,GENEtaBCD,Em


### Apply layer-by-layer corrections

In [19]:
# apply layer by layer
calib_eta = 'GENEtaBC'
calib_sample = 'photons-V9'
calib_pu = 'PU0'

for smp in samples:
    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    calib_table = calib_table_by_sample[smp.type]
    for tp in tps:
        for gen_sel in ['GENEtaBC', 'GENEtaD', 'GENEtaBCD']:
            for tp_sel in ['Em']:
                print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)

                sel_df = tree_data[(tree_data.smp == smp.type) &
                                   (tree_data.pu == smp.label) &
                                   (tree_data.tp == tp) &
                                   (tree_data.tp_sel == tp_sel) &
                                   (tree_data.gen_sel == gen_sel)]
                
                calib = calib_df[(calib_df.smp == calib_sample) &
                                 (calib_df.pu == calib_pu) &
                                   (calib_df.tp == tp) &
                                   (calib_df.tp_sel == tp_sel) &
                                   (calib_df.gen_sel == calib_eta)]
                calib_values = calib.layer_calib.values[0]
                # we set to 0 layer 0
                calib_values[0] = 0
#                 print calib_values
                tree_data.loc[sel_df.index, 'lcalib_energy'] = sel_df.cl_layer_energies.apply(lambda x: np.dot(x, calib_values))
                


------ SAMPLE: photons-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD

------ SAMPLE: ele-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD

------ SAMPLE: ele-V9, PU: PU200
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD


In [20]:
tree_data[tree_data.smp == 'ele-V9'].iloc[:10]

,cl_pt,cl_layer_energies,cl_eta,cl_abseta,gen_energy,gen_pt,uncalib_energy,uncalib_energy_nl0,smp,pu,tp,gen_sel,tp_sel,lcalib_energy
72294,23.877455,"[0.130386441946, 3.13072633743, 5.25537109375,...",-1.895183,1.895183,91.581436,26.919662,80.579428,80.449041,ele-V9,PU0,HMvDR,GENEtaBC,Em,81.434596
72295,31.666878,"[0.240713432431, 5.29911327362, 13.9726629257,...",-2.209442,2.209442,148.637558,32.241051,146.010079,145.769365,ele-V9,PU0,HMvDR,GENEtaBC,Em,146.618389
72296,88.012207,"[0.0, 2.34960699081, 14.6975412369, 44.6300354...",2.386415,2.386415,486.873138,88.793625,482.663007,482.663007,ele-V9,PU0,HMvDR,GENEtaBC,Em,482.805715
72297,79.160065,"[0.190564796329, 10.1607999802, 32.2133598328,...",1.875818,1.875818,265.762390,79.576866,264.396363,264.205798,ele-V9,PU0,HMvDR,GENEtaBC,Em,267.236815
72298,12.616868,"[0.19056481123, 3.14947319031, 7.13630628586, ...",-1.925085,1.925085,70.597046,20.165983,44.170708,43.980143,ele-V9,PU0,HMvDR,GENEtaBC,Em,45.583889
72299,90.945869,"[0.43127822876, 9.29844474792, 29.3700847626, ...",-2.325178,2.325178,486.232880,94.173981,468.221767,467.790488,ele-V9,PU0,HMvDR,GENEtaBC,Em,469.638042
72300,55.920006,"[0.305906653404, 11.0419025421, 29.407579422, ...",-1.800756,1.800756,182.924820,58.823948,173.883114,173.577208,ele-V9,PU0,HMvDR,GENEtaBC,Em,175.818864
72301,53.065533,"[0.0, 5.19288110733, 24.620880127, 46.80466842...",2.314971,2.314971,263.317505,51.512714,271.292116,271.292116,ele-V9,PU0,HMvDR,GENEtaBC,Em,270.791283
72302,47.017929,"[0.0, 3.44317388535, 18.8968391418, 40.0558013...",2.171712,2.171712,214.692398,48.314575,208.933797,208.933797,ele-V9,PU0,HMvDR,GENEtaBC,Em,209.265677
72303,86.341995,"[0.125371575356, 3.57440209389, 15.6598806381,...",2.040935,2.040935,343.810394,87.844429,334.325714,334.200343,ele-V9,PU0,HMvDR,GENEtaBC,Em,336.659511


### compute eta scale correction @ PU200

In [21]:
# compute the layer calibrations on photons

eta_calib_df = pd.DataFrame()
from sklearn.linear_model import LinearRegression
from scipy.optimize import least_squares
from scipy.optimize import lsq_linear

for smp in samples:
    if smp.label != 'PU200':
        continue

    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    calib_table = calib_table_by_sample[smp.type]
    for tp in tps:
        for gen_sel in ['GENEtaBCD']:
            for tp_sel in ['Em']:
                print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)

                sel_df = tree_data[(tree_data.smp == smp.type) &
                                   (tree_data.pu == smp.label) &
                                   (tree_data.tp == tp) &
                                   (tree_data.tp_sel == tp_sel) &
                                   (tree_data.gen_sel == gen_sel)]

                df = pd.DataFrame(columns=['eta_calib_0', 'eta_calib_1', 'smp', 'pu', 'tp', 'gen_sel', 'tp_sel'])

                # uncalib eta correction

                X = sel_df.cl_abseta.to_numpy().reshape(-1, 1)
                Y = sel_df.gen_energy - sel_df.uncalib_energy_nl0.to_numpy()
                corr_eta = LinearRegression(fit_intercept=True).fit(X, Y)
                print corr_eta.coef_[0], corr_eta.intercept_
    
                df.loc[0, 'eta_calib_0'] = (corr_eta.coef_[0], corr_eta.intercept_)

                Y = sel_df.gen_energy - sel_df.lcalib_energy.to_numpy()
                corr_eta = LinearRegression(fit_intercept=True).fit(X, Y)
                print corr_eta.coef_[0], corr_eta.intercept_

                df.loc[0, 'eta_calib_1'] = (corr_eta.coef_[0], corr_eta.intercept_)

                Y = sel_df.gen_pt - sel_df.lcalib_energy.to_numpy()/np.cosh(sel_df.cl_eta)
                corr_eta = LinearRegression(fit_intercept=True).fit(X, Y)
                print corr_eta.coef_[0], corr_eta.intercept_
                
                df.loc[:, 'smp'] = smp.type
                df.loc[:, 'pu'] = smp.label
                df.loc[:, 'tp'] = tp
                df.loc[:, 'gen_sel'] = gen_sel
                df.loc[:, 'tp_sel'] = tp_sel

                eta_calib_df = eta_calib_df.append(df, ignore_index=True)

                


------ SAMPLE: ele-V9, PU: PU200
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
-43.5312 82.5216
-46.6658 86.5143
-8.14586 15.5455
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD
-25.9991 56.5752
-33.4214 65.8564
-6.76129 14.0799


In [22]:
eta_calib_df

,eta_calib_0,eta_calib_1,smp,pu,tp,gen_sel,tp_sel
0,"(-43.5312, 82.5216)","(-46.6658, 86.5143)",ele-V9,PU200,HMvDR,GENEtaBCD,Em
1,"(-25.9991, 56.5752)","(-33.4214, 65.8564)",ele-V9,PU200,HMvDRshapeDr,GENEtaBCD,Em


In [23]:
# apply eta dependent corrections
eta_calib_eta = 'GENEtaBCD'
eta_calib_sample = 'ele-V9'
eta_calib_pu = 'PU200'

for smp in samples_ele_V9:
    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    calib_table = calib_table_by_sample[smp.type]
    for tp in tps:
        for gen_sel in ['GENEtaBC', 'GENEtaD', 'GENEtaBCD']:
            for tp_sel in ['Em']:
                print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)

                sel_df = tree_data[(tree_data.smp == smp.type) &
                                   (tree_data.pu == smp.label) &
                                   (tree_data.tp == tp) &
                                   (tree_data.tp_sel == tp_sel) &
                                   (tree_data.gen_sel == gen_sel)]
                
                calib = eta_calib_df[(eta_calib_df.smp == eta_calib_sample) &
                                 (eta_calib_df.pu == eta_calib_pu) &
                                   (eta_calib_df.tp == tp) &
                                   (eta_calib_df.tp_sel == tp_sel) &
                                   (eta_calib_df.gen_sel == eta_calib_eta)]
                calib_values = calib.eta_calib_0.values[0]
                print calib_values
                tree_data.loc[sel_df.index, 'eta_uncalib_energy_nl0'] = sel_df.uncalib_energy_nl0+calib_values[1]+sel_df.cl_abseta*calib_values[0]
                calib_values = calib.eta_calib_1.values[0]
                print calib_values
                tree_data.loc[sel_df.index, 'eta_lcalib_energy'] = sel_df.lcalib_energy+calib_values[1]+sel_df.cl_abseta*calib_values[0]

                # we set to 0 layer 0
                # calib_values[0] = 0
#                 print calib_values
                # tree_data.loc[sel_df.index, 'eta_lcalib_energy'] = sel_df.cl_layer_energies.apply(lambda x: np.dot(x, calib_values))
                


------ SAMPLE: ele-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
(-43.531178, 82.521622)
(-46.665813, 86.514267)
TP: HMvDR sel: Em, gen_sel: GENEtaD
(-43.531178, 82.521622)
(-46.665813, 86.514267)
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
(-43.531178, 82.521622)
(-46.665813, 86.514267)
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
(-25.999105, 56.575249)
(-33.421364, 65.856438)
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
(-25.999105, 56.575249)
(-33.421364, 65.856438)
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD
(-25.999105, 56.575249)
(-33.421364, 65.856438)

------ SAMPLE: ele-V9, PU: PU200
TP: HMvDR sel: Em, gen_sel: GENEtaBC
(-43.531178, 82.521622)
(-46.665813, 86.514267)
TP: HMvDR sel: Em, gen_sel: GENEtaD
(-43.531178, 82.521622)
(-46.665813, 86.514267)
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
(-43.531178, 82.521622)
(-46.665813, 86.514267)
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
(-25.999105, 56.575249)
(-33.421364, 65.856438)
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
(-25.999105, 

In [24]:
tree_data[tree_data.smp == 'ele-V9'].iloc[:10]

,cl_pt,cl_layer_energies,cl_eta,cl_abseta,gen_energy,gen_pt,uncalib_energy,uncalib_energy_nl0,smp,pu,tp,gen_sel,tp_sel,lcalib_energy,eta_uncalib_energy_nl0,eta_lcalib_energy
72294,23.877455,"[0.130386441946, 3.13072633743, 5.25537109375,...",-1.895183,1.895183,91.581436,26.919662,80.579428,80.449041,ele-V9,PU0,HMvDR,GENEtaBC,Em,81.434596,80.471128,79.508617
72295,31.666878,"[0.240713432431, 5.29911327362, 13.9726629257,...",-2.209442,2.209442,148.637558,32.241051,146.010079,145.769365,ele-V9,PU0,HMvDR,GENEtaBC,Em,146.618389,132.111376,130.027256
72296,88.012207,"[0.0, 2.34960699081, 14.6975412369, 44.6300354...",2.386415,2.386415,486.873138,88.793625,482.663007,482.663007,ele-V9,PU0,HMvDR,GENEtaBC,Em,482.805715,461.301175,457.955984
72297,79.160065,"[0.190564796329, 10.1607999802, 32.2133598328,...",1.875818,1.875818,265.762390,79.576866,264.396363,264.205798,ele-V9,PU0,HMvDR,GENEtaBC,Em,267.236815,265.070834,266.214484
72298,12.616868,"[0.19056481123, 3.14947319031, 7.13630628586, ...",-1.925085,1.925085,70.597046,20.165983,44.170708,43.980143,ele-V9,PU0,HMvDR,GENEtaBC,Em,45.583889,42.700534,42.262485
72299,90.945869,"[0.43127822876, 9.29844474792, 29.3700847626, ...",-2.325178,2.325178,486.232880,94.173981,468.221767,467.790488,ele-V9,PU0,HMvDR,GENEtaBC,Em,469.638042,449.094367,447.645977
72300,55.920006,"[0.305906653404, 11.0419025421, 29.407579422, ...",-1.800756,1.800756,182.924820,58.823948,173.883114,173.577208,ele-V9,PU0,HMvDR,GENEtaBC,Em,175.818864,177.709784,178.299364
72301,53.065533,"[0.0, 5.19288110733, 24.620880127, 46.80466842...",2.314971,2.314971,263.317505,51.512714,271.292116,271.292116,ele-V9,PU0,HMvDR,GENEtaBC,Em,270.791283,253.040323,249.275544
72302,47.017929,"[0.0, 3.44317388535, 18.8968391418, 40.0558013...",2.171712,2.171712,214.692398,48.314575,208.933797,208.933797,ele-V9,PU0,HMvDR,GENEtaBC,Em,209.265677,196.918241,194.435240
72303,86.341995,"[0.125371575356, 3.57440209389, 15.6598806381,...",2.040935,2.040935,343.810394,87.844429,334.325714,334.200343,ele-V9,PU0,HMvDR,GENEtaBC,Em,336.659511,327.877658,327.931880


In [25]:
#compute resolutions
# eta_uncalib_energy_nl0
# eta_lcalib_energy
tree_data['uncalib_res'] = tree_data.uncalib_energy_nl0 - tree_data.gen_energy
tree_data['lcalib_res'] = tree_data.lcalib_energy - tree_data.gen_energy
tree_data['eta_uncalib_res'] = tree_data.eta_uncalib_energy_nl0 - tree_data.gen_energy
tree_data['eta_lcalib_res'] = tree_data.eta_lcalib_energy - tree_data.gen_energy


In [26]:
hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)

for smp in samples:
    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    calib_table = calib_table_by_sample[smp.type]
    for tp in tps:
        for gen_sel in ['GENEtaBC', 'GENEtaD', 'GENEtaBCD']:
            for tp_sel in ['Em']:
                print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)
                hres = EnergyRes(name='{}_{}_{}_{}_{}'.format(smp.type, smp.label, tp, tp_sel, gen_sel))
    
                sel_df = tree_data[(tree_data.smp == smp.type) &
                   (tree_data.pu == smp.label) &
                   (tree_data.tp == tp) &
                   (tree_data.tp_sel == tp_sel) &
                   (tree_data.gen_sel == gen_sel)]

                
                
                hres.fill_df(sel_df)

                hplot.data = hplot.data.append({'sample': smp.type,
                                       'pu': smp.label,
                                       'tp': tp,
                                       'tp_sel': tp_sel,
                                       'gen_sel': gen_sel,
                                       'classtype': EnergyRes,
                                       'histo': HWrapper(hres),},
                                       ignore_index=True)
   


------ SAMPLE: photons-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD

------ SAMPLE: ele-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD

------ SAMPLE: ele-V9, PU: PU200
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD


In [27]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,photons-V9,PU0,HMvDR,Em,GENEtaBC,__main__.EnergyRes,<__main__.HWrapper object at 0x13e0a4e10>
1,photons-V9,PU0,HMvDR,Em,GENEtaD,__main__.EnergyRes,<__main__.HWrapper object at 0x13e0a4d10>
2,photons-V9,PU0,HMvDR,Em,GENEtaBCD,__main__.EnergyRes,<__main__.HWrapper object at 0x13e0a4e50>
3,photons-V9,PU0,HMvDRshapeDr,Em,GENEtaBC,__main__.EnergyRes,<__main__.HWrapper object at 0x13e0a4f90>
4,photons-V9,PU0,HMvDRshapeDr,Em,GENEtaD,__main__.EnergyRes,<__main__.HWrapper object at 0x13e0a4dd0>
5,photons-V9,PU0,HMvDRshapeDr,Em,GENEtaBCD,__main__.EnergyRes,<__main__.HWrapper object at 0x13e0a8150>
6,ele-V9,PU0,HMvDR,Em,GENEtaBC,__main__.EnergyRes,<__main__.HWrapper object at 0x13e0a4f10>
7,ele-V9,PU0,HMvDR,Em,GENEtaD,__main__.EnergyRes,<__main__.HWrapper object at 0x13e0a4ed0>
8,ele-V9,PU0,HMvDR,Em,GENEtaBCD,__main__.EnergyRes,<__main__.HWrapper object at 0x13e0a8410>
9,ele-V9,PU0,HMvDRshapeDr,Em,GENEtaBC,__main__.EnergyRes,<__main__.HWrapper object at 0x13e0a8250>


In [28]:
hplot.data.tp.unique()

array(['HMvDR', 'HMvDRshapeDr'], dtype=object)

In [29]:
hplot.labels_dict

{'Em': 'EGId',
 'EmPt10': 'EGId, p_{T}^{L1}>=10GeV',
 'EmPt20': 'EGId, p_{T}^{L1}>=20GeV',
 'EmPt30': 'EGId, p_{T}^{L1}>=30GeV',
 'GEN': '',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaBCDHBrem': '1.52 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5',
 'GENEtaBCDLBrem': '1.52 < |#eta^{GEN}| <= 2.8, f_{BREM} < 0.5',
 'GENEtaBCHBrem': '1.52 < |#eta^{GEN}| <= 2.4, f_{BREM} >= 0.5',
 'GENEtaBCLBrem': '1.52 < |#eta^{GEN}| <= 2.4, f_{BREM} < 0.5',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENEtaDHBrem': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5',
 'GENEtaDHBremPt10': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=10GeV',
 'GENEtaDHBremPt20': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=20GeV',
 'GENEtaDHBremPt30': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=30GeV',
 'GENEtaDHBremPt40': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=40GeV',
 'GENEtaDLBrem': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} < 0.5'

In [30]:
hfd = hplot.get_histo('pippo', 'ele-V9', ['PU0'], ['HMvDRcylind2p5'], 'Em', 'GENEtaBC')

print hfd[0]
print hfd[1]
print hfd[2]




No match found for: pu: ['PU0'], tp: ['HMvDRcylind2p5'], tp_sel: Em, gen_sel: GENEtaBC, classtype: pippo
None
None
None


In [30]:
print hfd[2]


None


## Comparison of the calibration sets for different selections

In [44]:
sample='photons-V9'

In [45]:
for tp in tps:
#     for tp_sel in tp_select[tp]:
        for gen_sel in ['GENEtaBC']:
            hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['all', 'Em'], gen_sel)
            draw(hsets, labels,  text=text, min_y=0., max_y=2.)

In [46]:
for tp in tps:
#     for tp_sel in tp_select[tp]:
#         for gen_sel in ['GENEtaBC']:
            hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['Em'], ['GENEtaBC', 'GENEtaD', 'GENEtaBCD'])
            draw(hsets, labels,  text=text, min_y=0., max_y=2.)

In [ ]:

hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5', 'HMvDRshape', 'HMvDRshapeDr'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=10., h_lines=[1.])

In [ ]:

hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind5', 'HMvDRcylind5Calib','HMvDRcylind5Calib1'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:

hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind10', 'HMvDRcylind10Calib','HMvDRcylind10Calib1'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRshape', 'HMvDRshapeCalib','HMvDRshapeCalib1'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:

hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind2p5Calib','HMvDRcylind2p5Calib1'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'Em', 'GENEtaBCPt10')
draw(hsets, labels,  text=text, min_y=0., max_y=10., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'Em', 'GENEtaBCPt40')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'all', 'GEN')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'all', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'all', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
for tp in tps:
    for gen_sel in ['GENEtaBC', 'GENEtaBCPt10', 'GENEtaBCPt20', 'GENEtaBCPt40']:
        hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['all', 'Em'], gen_sel)
        draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
for tp in tps:
    hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['Em'], ['GENEtaBC', 'GENEtaBCPt10', 'GENEtaBCPt20', 'GENEtaBCPt40'])
    draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
for tp in tps:
    hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['all'], ['GENEtaBC', 'GENEtaBCPt10', 'GENEtaBCPt20', 'GENEtaBCPt40'])
    draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind10'], ['all', 'Em'], ['GENEtaBC'])
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
len([1.527]+[1.]*12+[1.98])

## Energy Resolution

In [30]:
def gaussfit(name, project_hist):
    
    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()

#     print max_bin, max_value, rms_value
    def gausstail(x, p):
        #         // [Constant] * ROOT::Math::crystalball_function(x, [Alpha], [N], [Sigma], [Mean])
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', "gaus", -100, 100, 5)
#     fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-15.,max_value+15.)

    fitf.SetParameters(max_y,max_value, rms_value)
    draw([project_hist], labels=['fit'], text=name)
    stuff.append(fitf)
    project_hist.Draw("same")
#     c.Draw()
    print '   name: {}, int = {}, max_value = {}, RMS = {}'.format(name, project_hist.Integral(), max_value, rms_value)
    result = project_hist.Fit('gausstail','QERLS+')
    result.Print()
    print '   norm = {}, reso_mean = {}, reso_sigma = {}'.format(result.GetParams()[0], result.GetParams()[1], result.GetParams()[2])
#     func = project_hist.GetFunction("gaus")
    # print '   NDF = {}, chi2 = {}, prob = {}'.format(fitf.GetNDF(), fitf.GetChisquare(), fitf.GetProb())
    return result

def gausstailfit(name, project_hist):
    
    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()
#     print max_bin, max_value, rms_value
    def gausstail(x, p):
        #         // [Constant] * ROOT::Math::crystalball_function(x, [Alpha], [N], [Sigma], [Mean])
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', gausstail, -100, 100, 5)
    fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-15.,max_value+15.)

    fitf.SetParameters(max_y, max_value, rms_value, 1, 1)
    draw([project_hist], labels=['fit'], text=name)
    stuff.append(fitf)
    project_hist.Draw("same")
#     c.Draw()
    print '   name: {}, y_max = {}, max_value = {}, RMS = {}'.format(name, max_y, max_value, rms_value)
    result = project_hist.Fit('gausstail','QERLS+')
    result.Print()
    print '   norm = {}, reso_mean = {}, reso_sigma = {}, alpha = {}, n = {}'.format(result.GetParams()[0], result.GetParams()[1], result.GetParams()[2], result.GetParams()[3], result.GetParams()[4])
#     func = project_hist.GetFunction("gaus")
    # print '   NDF = {}, chi2 = {}, prob = {}'.format(fitf.GetNDF(), fitf.GetChisquare(), fitf.GetProb())
    return result

def gausstailfit_pt(name, project_hist):
    
    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()
#     print max_bin, max_value, rms_value
    def gausstail(x, p):
        #         // [Constant] * ROOT::Math::crystalball_function(x, [Alpha], [N], [Sigma], [Mean])
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', gausstail, 0, 2, 5)
    fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-15.,max_value+15.)

    fitf.SetParameters(max_y, max_value, rms_value, 1, 1)
    draw([project_hist], labels=['fit'], text=name)
    stuff.append(fitf)
    project_hist.Draw("same")
#     c.Draw()
    print '   name: {}, y_max = {}, max_value = {}, RMS = {}'.format(name, max_y, max_value, rms_value)
    result = project_hist.Fit('gausstail','QERLS+')
    result.Print()
    print '   norm = {}, reso_mean = {}, reso_sigma = {}, alpha = {}, n = {}'.format(result.GetParams()[0], result.GetParams()[1], result.GetParams()[2], result.GetParams()[3], result.GetParams()[4])
#     func = project_hist.GetFunction("gaus")
    # print '   NDF = {}, chi2 = {}, prob = {}'.format(fitf.GetNDF(), fitf.GetChisquare(), fitf.GetProb())
    return result



In [31]:
def computeEResolution(h2d_orig):
    h2d = h2d_orig.Clone()
    h2d.GetYaxis().SetRangeUser(-100, 100)
    x, y, ex_l, ex_h, ey_l, ey_h = [], [], [], [], [], []
    print '-----------------------'
    for x_bin_low, x_bin_high in [(3,6),(7,12),(13,23),(24,34),(35,49),(50,100)]:
        y_proj = h2d.ProjectionY(uuid.uuid4().hex[:6]+'_y', x_bin_low, x_bin_high)
        stuff.append(y_proj)
        x_low = h2d.GetXaxis().GetBinLowEdge(x_bin_low)
        x_high = h2d.GetXaxis().GetBinUpEdge(x_bin_high)
        print 'x_low: {} x_high: {}'.format(x_low, x_high)
        fit_result = gausstailfit(h2d_orig.GetName(), y_proj)

        h2d.SetAxisRange(x_low, x_high)
        x_mean = h2d.GetMean()
#         print 'mean: {}'.format(x_mean)
    #     x_value = h2d.GetXaxis().GetBinCenter(x_bin)
    #     x_err_min = x_value - h2d.GetXaxis().GetBinLowEdge(x_bin)
    #     x_err_plus = h2d.GetXaxis().GetBinUpEdge(x_bin) - x_value

        x.append(x_mean)
        ex_l.append(0)
        ex_h.append(0)

        y.append(fit_result.GetParams()[2]/x_mean)
        ey_l.append(0)
        ey_h.append(0)
    return x, y, ex_l, ex_h, ey_l, ey_h

def computeEResolutionMean(h2d_orig):
    h2d = h2d_orig.Clone()
    h2d.GetYaxis().SetRangeUser(-100, 100)
    x, y, ex_l, ex_h, ey_l, ey_h = [], [], [], [], [], []
    print '-----------------------'
    for x_bin_low, x_bin_high in [(3,6),(7,12),(13,23),(24,34),(35,49),(50,100)]:
        y_proj = h2d.ProjectionY(uuid.uuid4().hex[:6]+'_y', x_bin_low, x_bin_high)
        stuff.append(y_proj)
        x_low = h2d.GetXaxis().GetBinLowEdge(x_bin_low)
        x_high = h2d.GetXaxis().GetBinUpEdge(x_bin_high)
        print 'x_low: {} x_high: {}'.format(x_low, x_high)
        fit_result = gausstailfit(h2d_orig.GetName(), y_proj)

        h2d.SetAxisRange(x_low, x_high)
        x_mean = h2d.GetMean()
#         print 'mean: {}'.format(x_mean)
    #     x_value = h2d.GetXaxis().GetBinCenter(x_bin)
    #     x_err_min = x_value - h2d.GetXaxis().GetBinLowEdge(x_bin)
    #     x_err_plus = h2d.GetXaxis().GetBinUpEdge(x_bin) - x_value

        x.append(x_mean)
        ex_l.append(0)
        ex_h.append(0)

        y.append(fit_result.GetParams()[1])
        ey_l.append(0)
        ey_h.append(0)
    return x, y, ex_l, ex_h, ey_l, ey_h



In [32]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU0'], ['HMvDREmInt', 'HMvDREmIntMerged'], 'Em', 'GENEtaD')
draw([his.h_energyResVenergy for his in hsets], labels,  text=text, options='COLZ', min_y=-100, max_y=100)
draw([his.h_energyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist')

No match found for: pu: ['PU0'], tp: ['HMvDREmInt', 'HMvDREmIntMerged'], tp_sel: Em, gen_sel: GENEtaD, classtype: __main__.EnergyRes


TypeError: 'NoneType' object is not iterable

In [26]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU0', 'PU200'], ['HMvDREmInt'], 'Em', 'GENEtaBC')
draw([his.h_energyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist', norm=True)

<ROOT.TCanvas object ("c_e5cbb3") at 0x7f919e2f5810>

In [27]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU0', 'PU200'], ['HMvDRshapeDr'], 'Em', 'GENEtaBC')
draw([his.h_energyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist', norm=True)

<ROOT.TCanvas object ("c_4dadfd") at 0x7f919e4bdd50>

In [28]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU0', 'PU200'], ['HMvDRshapeDrCalib'], 'Em', 'GENEtaBC')
draw([his.h_energyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist', norm=True)

<ROOT.TCanvas object ("c_ad17fe") at 0x7f919f5439d0>

In [29]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU0'], ['HMvDR', 'HMvDREmInt', 'HMvDRshapeDr'], 'Em', 'GENEtaBC')
draw([his.h_energyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist', norm=True)

<ROOT.TCanvas object ("c_ebc928") at 0x7f919e4c1720>

In [30]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU0'], ['HMvDR', 'HMvDREmInt', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], 'Em', 'GENEtaBC')
draw([his.h_energyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist', norm=True)

<ROOT.TCanvas object ("c_f0daea") at 0x7f919d34ed90>

In [34]:

hsets, labels, text = hplot.get_histo(EnergyRes, ['photons-V9'], ['PU0'], ['HMvDRshapeDr'], 'Em', 'GENEtaBC')
hsets_new = []
labels_new = []


hsets_new.append(hsets[0].h_energyResp)
labels_new.append('un-calib')                                             

hsets_new.append(hsets[0].h_lCalibEnergyResp)
labels_new.append('layer-weights')                                             




draw(hsets_new, labels_new,  text=text, options='hist', norm=True)


<ROOT.TCanvas object ("c_6e5928") at 0x7fbbc1c27040>

In [35]:

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDRshapeDr'], 'Em', 'GENEtaBC')
hsets_new = []
labels_new = []


hsets_new.append(hsets[0].h_energyResp)
labels_new.append('un-calib')                                             

hsets_new.append(hsets[0].h_lCalibEnergyResp)
labels_new.append('layer-weights')                                             

hsets_new.append(hsets[0].h_etaEnergyResp)
labels_new.append('eta-corr')                                             

hsets_new.append(hsets[0].h_etalCalibEnergyResp)
labels_new.append('layer-weights+eta-corr')                                             



draw(hsets_new, labels_new,  text=text, options='hist', norm=True)


<ROOT.TCanvas object ("c_96aef6") at 0x7fbbc1c80cd0>

In [36]:

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDRshapeDr'], 'Em', 'GENEtaBCD')
hsets_new = []
labels_new = []


hsets_new.append(hsets[0].h_energyResp)
labels_new.append('un-calib')                                             

hsets_new.append(hsets[0].h_lCalibEnergyResp)
labels_new.append('layer-weights')                                             

hsets_new.append(hsets[0].h_etaEnergyResp)
labels_new.append('eta-corr')                                             

hsets_new.append(hsets[0].h_etalCalibEnergyResp)
labels_new.append('layer-weights+eta-corr')                                             



draw(hsets_new, labels_new,  text=text, options='hist', norm=True)


<ROOT.TCanvas object ("c_26e1e7") at 0x7fbbc169eb10>

In [34]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDR','HMvDRshapeDr'], 'Em', 'GENEtaBC')
draw([his.h_energyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist', norm=True)
draw([his.h_lCalibEnergyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist', norm=True)
draw([his.h_etaEnergyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist', norm=True)
draw([his.h_etalCalibEnergyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist', norm=True)


<ROOT.TCanvas object ("c_a39102") at 0x7f8ce46dd180>

In [35]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDR','HMvDRshapeDr'], 'Em', 'GENEtaD')
draw([his.h_energyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist', norm=True)
draw([his.h_lCalibEnergyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist', norm=True)
draw([his.h_etaEnergyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist', norm=True)
draw([his.h_etalCalibEnergyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist', norm=True)


<ROOT.TCanvas object ("c_fb2c64") at 0x7f8ce46e2330>

In [19]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], 'Em', 'GENEtaD')
draw([his.h_energyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist', norm=True)

<ROOT.TCanvas object ("c_f65cd8") at 0x7fcf36e3a190>

In [20]:

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9', 'ele-V10'], ['PU0'], ['HMvDR'], 'Em', 'GENEtaBC')
draw([his.h_energyResVenergy for his in hsets], labels,  text=text, options='COLZ', min_y=-100, max_y=100)
draw([his.h_energyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist')

<ROOT.TCanvas object ("c_2e7459") at 0x7fcf36a921b0>

In [21]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9', 'ele-V10'], ['PU0'], ['HMvDRCalib'], 'Em', 'GENEtaBC')
draw([his.h_energyResVenergy for his in hsets], labels,  text=text, options='COLZ', min_y=-100, max_y=100)
draw([his.h_energyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist')

<ROOT.TCanvas object ("c_bc696f") at 0x7fcf36bb2dc0>

In [ ]:

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9', 'ele-V10'], ['PU0'], ['HMvDRshapeDr'], 'Em', 'GENEtaBC')
draw([his.h_energyResVenergy for his in hsets], labels,  text=text, options='COLZ', min_y=-100, max_y=100)
draw([his.h_energyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist')

In [ ]:

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9', 'ele-V10'], ['PU0'], ['HMvDRshapeDrCalib'], 'Em', 'GENEtaBC')
draw([his.h_energyResVenergy for his in hsets], labels,  text=text, options='COLZ', min_y=-100, max_y=100)
draw([his.h_energyResVenergy.ProjectionY(uuid.uuid4().hex[:6]) for his in hsets], labels,  text=text, options='hist')

In [39]:
# from scipy.stats import crystalball

# sel_df = tree_data[(tree_data.smp == 'ele-V9') &
#                    (tree_data.pu == 'PU0') &
#                    (tree_data.tp == 'HMvDR') &
#                    (tree_data.tp_sel == 'Em') &
#                    (tree_data.gen_sel == 'GENEtaBC')]
# pt_sel = sel_df[(sel_df.gen_energy > 20) & (sel_df.gen_energy <= 60)]     

# # 'uncalib_res'
# # 'lcalib_res'
# # 'eta_uncalib_res'
# # 'eta_lcalib_res'
# pt_sel['uncalib_res']


['__add__', '__class__', '__contains__', '__delattr__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__getslice__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'count', 'index']
(0.10632148653461404, 103.19715377137854, -0.24076577988642567, 0.9798862482670585)


In [38]:
for sample in ['photons-V9']:
    for tp in tps:
        for pu in ['PU0']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaBC', debug=False)
    #         print dir(hsets[0])
            draw([his.h_energyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_energyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 133.0, max_value = -2.5, RMS = 2.60651353698
   norm = 134.171091397, reso_mean = -1.89302117768, reso_sigma = 2.44786232252, alpha = 2.5320893891, n = 2.66047819369
x_low: 60.0 x_high: 120.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 191.0, max_value = -2.5, RMS = 3.900344414
   norm = 171.966668083, reso_mean = -1.98885312326, reso_sigma = 3.49094795128, alpha = 2.18810686942, n = 3.12766750364
x_low: 120.0 x_high: 230.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 215.0, max_value = -1.5, RMS = 6.56042821727
   norm = 206.809612734, reso_mean = -1.50365475577, reso_sigma = 5.52360918774, alpha = 2.58793346349, n = 1.77534273818
x_low: 230.0 x_high: 340.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 168.0, max_value = -0.5, RMS = 8.37936041145
   norm = 157.050400025, reso_mean = 0.913187345002, res

In [37]:
for sample in ['photons-V9']:
    for tp in tps:
        for pu in ['PU0']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaBC', debug=False)
    #         print dir(hsets[0])
            draw([his.h_lCalibEnergyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_lCalibEnergyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 138.0, max_value = -0.5, RMS = 2.60789760954
   norm = 132.931866371, reso_mean = -1.72249913581, reso_sigma = 2.47459047396, alpha = 2.72119267849, n = 2.35463342633
x_low: 60.0 x_high: 120.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 196.0, max_value = -2.5, RMS = 3.90634555285
   norm = 170.277671324, reso_mean = -1.90322804335, reso_sigma = 3.52850823913, alpha = 2.20567237721, n = 3.24271579543
x_low: 120.0 x_high: 230.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 220.0, max_value = 0.5, RMS = 6.48956253899
   norm = 207.642671661, reso_mean = -1.4130469302, reso_sigma = 5.50235216027, alpha = 2.5694751814, n = 1.87899601796
x_low: 230.0 x_high: 340.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 169.0, max_value = -0.5, RMS = 8.07717971814
   norm = 159.414013774, reso_me

In [33]:
for sample in ['ele-V9']:
    for tp in tps:
        for pu in ['PU0', 'PU200']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaBC', debug=False)
    #         print dir(hsets[0])
            draw([his.h_lCalibEnergyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_lCalibEnergyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 253.0, max_value = -4.5, RMS = 6.93604225871
   norm = 245.143273599, reso_mean = -3.75474534562, reso_sigma = 2.87446793324, alpha = 0.403974318258, n = 2610997.65726
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 310.0, max_value = -1.5, RMS = 10.0878584251
   norm = 303.608554489, reso_mean = -3.21363099411, reso_sigma = 3.88428194407, alpha = 0.408121502175, n = 111.360012343
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 441.0, max_value = -5.5, RMS = 11.4409850948
   norm = 453.00219805, reso_mean = -3.39578012631, reso_sigma = 5.97270997204, alpha = 0.660547288746, n = 14.491716033
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 273.0, max_value = -2.5, RMS = 11.1585724352
   norm = 280.033369075, reso_mean = -2.6634

   norm = 235.760914677, reso_mean = -3.65262736787, reso_sigma = 2.89926092381, alpha = 0.389794784722, n = 2130908.83439
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 299.0, max_value = -1.5, RMS = 10.783468155
   norm = 292.117609333, reso_mean = -2.82823123055, reso_sigma = 3.80908017994, alpha = 0.378580732643, n = 58.2558647273
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 442.0, max_value = -3.5, RMS = 12.4411192919
   norm = 435.641647155, reso_mean = -2.94574152319, reso_sigma = 5.89583340252, alpha = 0.612539859786, n = 11.2769023935
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 263.0, max_value = -0.5, RMS = 12.1315075408
   norm = 269.046055832, reso_mean = -1.920810887, reso_sigma = 7.55915583825, alpha = 0.723461149304, n = 72.3632055967
x_low: 340.0 x_high: 490.0
   name: ele-V9_PU0_HMvDRshapeDr_Em_GE

In [34]:
for sample in ['ele-V9']:
    for tp in tps:
        for pu in ['PU0', 'PU200']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaBC', debug=False)
    #         print dir(hsets[0])
            draw([his.h_etaEnergyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_etaEnergyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_etaEnergyResVenergy, y_max = 96.0, max_value = -3.5, RMS = 12.1423926944
   norm = 90.942018838, reso_mean = -8.66725266883, reso_sigma = 12.1424182202, alpha = 5.68005090492, n = 6.26985488312
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_etaEnergyResVenergy, y_max = 137.0, max_value = -10.5, RMS = 13.810614405
   norm = 139.860299765, reso_mean = -8.96910884788, reso_sigma = 13.0108045219, alpha = 1.62223843702, n = 52.7076729499
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_etaEnergyResVenergy, y_max = 218.0, max_value = -14.5, RMS = 14.9985313697
   norm = 235.003384246, reso_mean = -8.86435653933, reso_sigma = 14.5280586955, alpha = 2.67259749031, n = 1.6928595843
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_etaEnergyResVenergy, y_max = 172.0, max_value = -9.5, RMS = 14.7009764044
   norm = 170.489709117, reso_mean = -10.8929770511, reso_s

   norm = 191.435860941, reso_mean = -1.27833423921, reso_sigma = 8.53046075094, alpha = 0.87227371269, n = 2751544.70726
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_etaEnergyResVenergy, y_max = 283.0, max_value = 1.5, RMS = 13.1934282135
   norm = 314.375847746, reso_mean = -1.97031546903, reso_sigma = 9.63301970396, alpha = 0.872510804739, n = 1355930.90066
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_etaEnergyResVenergy, y_max = 203.0, max_value = -0.5, RMS = 12.8723599061
   norm = 216.208451275, reso_mean = -4.91563932001, reso_sigma = 10.4706148141, alpha = 1.04106579119, n = 10928450.9286
x_low: 340.0 x_high: 490.0
   name: ele-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_etaEnergyResVenergy, y_max = 118.0, max_value = -9.5, RMS = 12.3222850502
   norm = 107.216454082, reso_mean = -10.5069464295, reso_sigma = 9.78649118191, alpha = 0.968573195072, n = 2865985.93239
x_low: 490.0 x_high: 1000.0
   name: ele-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_

In [35]:
for sample in ['ele-V9']:
    for tp in tps:
        for pu in ['PU0', 'PU200']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaBC', debug=False)
    #         print dir(hsets[0])
            draw([his.h_energyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_energyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 248.0, max_value = -3.5, RMS = 6.88941618905
   norm = 250.87471481, reso_mean = -4.26520633327, reso_sigma = 2.73562566884, alpha = 0.386165331543, n = 1467020.75793
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 316.0, max_value = -4.5, RMS = 10.0997799838
   norm = 305.82832244, reso_mean = -4.13500397367, reso_sigma = 3.83255639688, alpha = 0.40547727461, n = 77.129911088
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 442.0, max_value = -5.5, RMS = 11.6866696849
   norm = 456.996763419, reso_mean = -4.46518663397, reso_sigma = 5.85755421104, alpha = 0.656436858704, n = 11.4613686097
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 284.0, max_value = -5.5, RMS = 11.2649289174
   norm = 280.344687496, reso_mean = -3.90862927995, reso_sigma = 7.66

   norm = 298.950625228, reso_mean = -5.07228570927, reso_sigma = 3.63831280602, alpha = 0.364153674455, n = 64.9504414335
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_energyResVenergy, y_max = 448.0, max_value = -6.5, RMS = 12.4768959525
   norm = 449.199140999, reso_mean = -6.66337575478, reso_sigma = 5.54212547285, alpha = 0.577386870933, n = 10.4784877261
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_energyResVenergy, y_max = 277.0, max_value = -8.5, RMS = 11.9679525832
   norm = 274.068411512, reso_mean = -7.68276370345, reso_sigma = 7.30077728663, alpha = 0.684140987953, n = 718.162927244
x_low: 340.0 x_high: 490.0
   name: ele-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_energyResVenergy, y_max = 131.0, max_value = -4.5, RMS = 11.7906800626
   norm = 114.399045539, reso_mean = -9.25704108035, reso_sigma = 9.00821390693, alpha = 0.901430464273, n = -3132327.80009
x_low: 490.0 x_high: 1000.0
   name: ele-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_energ

In [39]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_energyResVenergy_meanEReso for his in hsets], labels, text=text, options='COLZ', min_y=-50, max_y=50)


In [40]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU200', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_energyResVenergy_meanEReso for his in hsets], labels, text=text, options='COLZ', min_y=-50, max_y=50)


In [41]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_etaEnergyResVenergy_meanEReso for his in hsets], labels, text=text, options='COLZ', min_y=-50, max_y=50)


In [42]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU200', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_etaEnergyResVenergy_meanEReso for his in hsets], labels, text=text, options='COLZ', min_y=-50, max_y=50)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9', 'ele-V10'], 'PU0', ['HMvDRCalib'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_meanEReso for his in hsets], labels, text=text, options='COLZ', min_y=-50, max_y=50)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9', 'ele-V10'], 'PU0', ['HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_meanEReso for his in hsets], labels, text=text, options='COLZ', min_y=-50, max_y=50)


In [ ]:
for sample in ['ele-V9', 'ele-V10']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


In [43]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


In [44]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU200', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


In [45]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_etaEnergyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


In [46]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU200', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_etaEnergyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


The history saving thread hit an unexpected error (OperationalError('unable to open database file',)).History will not be written to the database.

Exception in thread IPythonHistorySavingThread:
Traceback (most recent call last):
  File "/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/threading.py", line 801, in __bootstrap_inner
  File "<decorator-gen-24>", line 2, in run
  File "/Users/cerminar/.virtualenvs/HGCTPGPerformance/lib/python2.7/site-packages/IPython/core/history.py", line 60, in needs_sqlite
  File "/Users/cerminar/.virtualenvs/HGCTPGPerformance/lib/python2.7/site-packages/IPython/core/history.py", line 843, in run
  File "/Users/cerminar/.virtualenvs/HGCTPGPerformance/lib/python2.7/site-packages/ipykernel/iostream.py", line 376, in write
  File "/Users/cerminar/.virtualenvs/HGCTPGPerformance/lib/python2.7/site-packages/ipykernel/iostream.py", line 203, in schedule
  File "/Users/cerminar/.virtualenvs/HGCTPGPerformance/lib/python2.7/site-packages/ipykernel/iostream.py", line 101, in _event_pipe
  File "/Users/cerminar/.virtualenvs/HGCTPGPerformance/lib/python2.7/site-packages/zmq/sugar/conte

In [40]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU200', ['HMvDRshapeDr'], ['Em'], 'GENEtaBC', debug=False)
    hset_new = []
    labels_new = []

    hset_new.append(hsets[0].g_energyResVenergy_sigmaEOE)
    labels_new.append('no corr.')

    hset_new.append(hsets[0].g_etaEnergyResVenergy_sigmaEOE)
    labels_new.append('eta corr.')

    hset_new.append(hsets[0].g_lCalibEnergyResVenergy_sigmaEOE)
    labels_new.append('layer calib.')

    
    draw(hset_new, labels_new, options='L', text=text,  min_y=-0, max_y=0.1)


In [43]:
for sample in ['photons-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshapeDr'], ['Em'], 'GENEtaBC', debug=False)
    hset_new = []
    labels_new = []

    hset_new.append(hsets[0].g_energyResVenergy_meanEReso)
    labels_new.append('no corr.')

    hset_new.append(hsets[0].g_lCalibEnergyResVenergy_meanEReso)
    labels_new.append('layer calib.')

    
    draw(hset_new, labels_new, options='L', text=text,  min_y=-10, max_y=10)


In [41]:
for sample in ['photons-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshapeDr'], ['Em'], 'GENEtaBC', debug=False)
    hset_new = []
    labels_new = []

    hset_new.append(hsets[0].g_energyResVenergy_sigmaEOE)
    labels_new.append('no corr.')

    hset_new.append(hsets[0].g_lCalibEnergyResVenergy_sigmaEOE)
    labels_new.append('layer calib.')

    
    draw(hset_new, labels_new, options='L', text=text,  min_y=-0, max_y=0.1)


In [41]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample,[ 'PU0', 'PU200'], ['HMvDRshapeDr'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9', 'ele-V10'], 'PU0', ['HMvDRCalib'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9', 'ele-V10'], 'PU0', ['HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshape','HMvDRcylind2p5','HMvDRcylind5', 'HMvDRshapeDr'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshapeCalib','HMvDRcylind2p5Calib','HMvDRcylind5Calib', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshapeCalib1','HMvDRcylind2p5Calib1','HMvDRcylind5Calib1', 'HMvDRcylind10Calib1'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
    

In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRfixedCalib1','HMvDRfixed'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRcylind2p5','HMvDRcylind2p5Calib1'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
